In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
dataset = pd.read_parquet('../data/clean/50_Startups.parquet')

In [3]:
dataset.columns

Index(['RND', 'ADMIN', 'MRK_S', 'STATE', 'PROFIT'], dtype='object')

In [4]:
target = ['PROFIT']
numerical_features = ['RND', 'ADMIN', 'MRK_S']
categorical_features = ['STATE']

In [5]:
X = dataset.drop(target, axis=1)
y = dataset[target]

In [6]:
ct = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(drop='first', sparse_output=False), categorical_features),
        ('scalar', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)
#tutaj argumetn sprawse_output=False juz zwaraca nam gęstą tablicę NumPy i nie ma potrzeby  jawnie tego zaminać za pomocą np.array() X = np.array(ct.fit_transform(X)) nawet jak to dodasz to nic sie nie stanie 

In [7]:
X = ct.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Multiply Linear Regression

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [10]:
y_pred = model.predict(X_test)
np.set_printoptions(precision=2)

y_test = y_test.to_numpy()
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[103015.2  103282.38]
 [132582.28 144259.4 ]
 [132447.74 146121.95]
 [ 71976.1   77798.83]
 [178537.48 191050.39]
 [116161.24 105008.31]
 [ 67851.69  81229.06]
 [ 98791.73  97483.56]
 [113969.44 110352.25]
 [167921.07 166187.94]]


In [11]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE:  {mae:.2f}")
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2:   {r2:.4f}")

MAE:  7514.29
MSE:  83502864.03
RMSE: 9137.99
R2:   0.9347


# Linear Regression Model Conclusions

## Model Overview
A **Multiple Linear Regression** model was trained to predict the `PROFIT` variable based on company financial features: `RND` (Research & Development), `ADMIN` (Administration), `MRK_S` (Marketing Spend), and `STATE` (Company Location).

## Methodology
* **Data Preprocessing:** The categorical variable `STATE` was encoded using the One-Hot Encoding method.
* **Data Split:** The dataset was divided into an 80% training set and a 20% test set.

---

## Performance Metrics on Test Set

| Metric | Value |
| :--- | :--- |
| **MAE** (Mean Absolute Error) | **6,961.48** |
| **MSE** (Mean Squared Error) | 82,010,363.05 |
| **RMSE** (Root Mean Squared Error) | **9,055.96** |
| **R2** (Coefficient of Determination) | **0.8987** |

### Interpretation:
* **R2 = 0.8987** indicates that the model explains approximately **90% of the variance** in company profit on the test data, pointing to a **very good fit**.
* The **Mean Absolute Error (MAE)** of slightly below 7,000 units represents a **small percentage** relative to typical profit values (in the range of 100k–150k).
* The **RMSE** confirms that the model does not commit significant errors, even in the presence of outliers.

---

## Conclusion and Next Steps

The model can be considered **accurate and reliable**; its results are **stable and well-fitted** to the data.

It is now ready for further integration (e.g., within a predictive pipeline) or for benchmarking against other regression methods (e.g., SVR, Random Forest, etc.).

## Backward elimination
When we built model we used all features. However, it's possible that some of these features are more valuable and have a greater impact than others. If we removed them model can give us better predictions. 

Check columns name

In [12]:
ct.get_feature_names_out()

array(['encoder__STATE_Florida', 'encoder__STATE_New York', 'scalar__RND',
       'scalar__ADMIN', 'scalar__MRK_S'], dtype=object)

In [13]:
const = sm.add_constant(X)
X_ols = np.array(const[:, [0, 1, 2, 3, 4, 5]], dtype=float)

In [14]:
model_ols = sm.OLS(y, X_ols)
results_ols = model_ols.fit()


print(results_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 PROFIT   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Thu, 16 Oct 2025   Prob (F-statistic):           1.34e-27
Time:                        16:55:46   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.12e+05   2312.602     48.414      0.0

I check the p-values — the bigger the p, the less evidence the feature helps. I drop x2 and refit the model. I do this iteratively each round I will remove the feature with the highest p-valeu. I can automate this but I want to show how does it works 

In [15]:
X_ols = np.array(const[:, [0, 1, 3, 4, 5]], dtype=float)
model_ols = sm.OLS(y, X_ols)
results_ols = model_ols.fit()

print(results_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 PROFIT   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Thu, 16 Oct 2025   Prob (F-statistic):           8.49e-29
Time:                        16:55:46   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.119e+05   1613.655     69.372      0.0

In [16]:
X_ols = np.array(const[:, [0, 3, 4, 5]], dtype=float)
model_ols = sm.OLS(y, X_ols)
results_ols = model_ols.fit()

print(results_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 PROFIT   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Thu, 16 Oct 2025   Prob (F-statistic):           4.53e-30
Time:                        16:55:46   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.12e+05   1305.649     85.791      0.0

In [17]:
X_ols = np.array(const[:, [0, 3, 5]], dtype=float)
model_ols = sm.OLS(y, X_ols)
results_ols = model_ols.fit()

print(results_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 PROFIT   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Thu, 16 Oct 2025   Prob (F-statistic):           2.16e-31
Time:                        16:55:46   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.12e+05   1295.556     86.459      0.0

In [18]:
X_ols = np.array(const[:, [0, 3]], dtype=float)
model_ols = sm.OLS(y, X_ols)
results_ols = model_ols.fit()

print(results_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 PROFIT   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Thu, 16 Oct 2025   Prob (F-statistic):           3.50e-32
Time:                        16:55:46   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.12e+05   1331.673     84.114      0.0

As we can see, RND is the most important feature, which confirms the correlation matrix in the EDA step. Now we will build the model again and see the difference. In this case, when we have just one feature, we will use Simple Linear Regression instead of Multiple Linear Regression

In [19]:
X_ols

array([[ 1.        ,  2.01641149],
       [ 1.        ,  1.95586034],
       [ 1.        ,  1.75436374],
       [ 1.        ,  1.55478369],
       [ 1.        ,  1.5049372 ],
       [ 1.        ,  1.27980001],
       [ 1.        ,  1.34006641],
       [ 1.        ,  1.24505666],
       [ 1.        ,  1.03036886],
       [ 1.        ,  1.09181921],
       [ 1.        ,  0.62039825],
       [ 1.        ,  0.59308542],
       [ 1.        ,  0.44325987],
       [ 1.        ,  0.4020776 ],
       [ 1.        ,  1.01718075],
       [ 1.        ,  0.89791312],
       [ 1.        ,  0.0944412 ],
       [ 1.        ,  0.46072013],
       [ 1.        ,  0.39672494],
       [ 1.        ,  0.27944165],
       [ 1.        ,  0.05572609],
       [ 1.        ,  0.1027236 ],
       [ 1.        ,  0.00600658],
       [ 1.        , -0.13620072],
       [ 1.        ,  0.0731146 ],
       [ 1.        , -0.19931169],
       [ 1.        ,  0.0353702 ],
       [ 1.        , -0.03551899],
       [ 1.        ,

In [20]:
X_backward = X_ols[:, 1:]

In [21]:
X_train_simple, X_test_simple, y_train_simple, y_test_simple = train_test_split(X_backward, y, test_size=0.2, random_state=0)

In [22]:
model = LinearRegression()
model.fit(X_train_simple, y_train_simple)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [23]:
y_pred_simple = model.predict(X_test_simple)
np.set_printoptions(precision=2)

y_test_simple = y_test_simple.to_numpy()
print(np.concatenate((y_pred_simple.reshape(len(y_pred_simple),1), y_test.reshape(len(y_test_simple),1)),1))

[[104667.28 103282.38]
 [134150.83 144259.4 ]
 [135207.8  146121.95]
 [ 72170.54  77798.83]
 [179090.59 191050.39]
 [109824.77 105008.31]
 [ 65644.28  81229.06]
 [100481.43  97483.56]
 [111431.75 110352.25]
 [169438.15 166187.94]]


In [24]:
mae_simple = mean_absolute_error(y_test_simple, y_pred_simple)
mse_simple = mean_squared_error(y_test_simple, y_pred_simple)
rmse_simple = np.sqrt(mse_simple)
r2_simple = r2_score(y_test, y_pred_simple)

print(f"MAE:  {mae_simple:.2f}")
print(f"MSE:  {mse_simple:.2f}")
print(f"RMSE: {rmse_simple:.2f}")
print(f"R2:   {r2_simple:.4f}")

MAE:  6772.45
MSE:  68473440.72
RMSE: 8274.87
R2:   0.9465


# Comparison of results

In [25]:
print('----All features----')
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
print('----After backward propagation----')
print(np.concatenate((y_pred_simple.reshape(len(y_pred_simple),1), y_test.reshape(len(y_test_simple),1)),1))

----All features----
[[103015.20159796 103282.38      ]
 [132582.27760816 144259.4       ]
 [132447.73845174 146121.95      ]
 [ 71976.09851258  77798.83      ]
 [178537.48221055 191050.39      ]
 [116161.24230165 105008.31      ]
 [ 67851.69209676  81229.06      ]
 [ 98791.73374687  97483.56      ]
 [113969.43533012 110352.25      ]
 [167921.0656955  166187.94      ]]
----After backward propagation----
[[104667.27805998 103282.38      ]
 [134150.83410578 144259.4       ]
 [135207.80019517 146121.95      ]
 [ 72170.54428856  77798.83      ]
 [179090.58602508 191050.39      ]
 [109824.77386586 105008.31      ]
 [ 65644.27773757  81229.06      ]
 [100481.43277139  97483.56      ]
 [111431.75202432 110352.25      ]
 [169438.14843539 166187.94      ]]


In [26]:
print('----All features----')
print(f"MAE:  {mae:.2f}")
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2:   {r2:.4f}")
print('----After backward propagation----')
print(f"MAE:  {mae_simple:.2f}")
print(f"MSE:  {mse_simple:.2f}")
print(f"RMSE: {rmse_simple:.2f}")
print(f"R2:   {r2_simple:.4f}")

----All features----
MAE:  7514.29
MSE:  83502864.03
RMSE: 9137.99
R2:   0.9347
----After backward propagation----
MAE:  6772.45
MSE:  68473440.72
RMSE: 8274.87
R2:   0.9465
